### Runs an extended Kalman filter using IHME SEIIR predictions along with measurement data of confirmed Covid-19 case counts (from New York Times data) and Facebook symptom data (loss of smell/taste, from Covid-19 Symptom Challenge) to generate updated 7-day predictions of case counts for counties in New York State.

Developed by the University of Washington team of Les Atlas, Abraham Flaxman and Michael Rhoads.

S - Susceptible
E - Exposed
I1 - Presymptomatic
I2 - Symptomatic
R - Recovered

In [1]:
#%load_ext autoreload
#%autoreload

In [2]:
#%reset

In [3]:
import math
import datetime

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

import data_sets
import seiir_compartmental

In [16]:
# functions to support the Kalman filtering
def get_predicts_prior(day, seiir):
    x_hat = np.array([[seiir['S'].loc[day]],
                      [seiir['E'].loc[day]],
                      [seiir['I1'].loc[day]],
                      [seiir['I2'].loc[day]],
                      [seiir['R'].loc[day]]])

    beta_k = seiir['beta'].loc[day]

    return x_hat, beta_k


def step_seiir(x_hat, constants, beta_k, days=7):
    s_dict = {'S': x_hat[0, 0],
              'E': x_hat[1, 0],
              'I1': x_hat[2, 0],
              'I2': x_hat[3, 0],
              'R': x_hat[4, 0]}

    s = pd.Series(s_dict)

    for i in range(days):
        infectious = s.loc['I1'] + s.loc['I2']
        s = seiir_compartmental.compartmental_covid_step(s, s.sum(),
                                                         infectious,
                                                         constants['alpha'],
                                                         beta_k,
                                                         constants['gamma1'],
                                                         constants['gamma2'],
                                                         constants['sigma'],
                                                         constants['theta'])
    x_hat_future_prior = np.array([[s.loc['S']],
                                   [s.loc['E']],
                                   [s.loc['I1']],
                                   [s.loc['I2']],
                                   [s.loc['R']]])

    return x_hat_future_prior


def predict_step(x_hat_k1_prior, P, Q, beta_k, constants):
    S = x_hat_k1_prior[0, 0]
    E = x_hat_k1_prior[1, 0]
    I1 = x_hat_k1_prior[2, 0]
    I2 = x_hat_k1_prior[3, 0]
    R = x_hat_k1_prior[4, 0]
    N = S + E + I1 + I2 + R
    alpha = constants['alpha']
    sigma = constants['sigma']
    gamma1 = constants['gamma1']
    gamma2 = constants['gamma2']

    part_f_S = np.array([[-beta_k * math.pow(I1 + I2, alpha) / N],
                         [beta_k * math.pow(I1 + I2, alpha) / N],
                         [0],
                         [0],
                         [0]])

    part_f_E = np.array([[0],
                         [-sigma],
                         [sigma],
                         [0],
                         [0]])

    part_f_I1 = np.array([[-alpha * beta_k * S * math.pow(I1+I2, alpha-1) / N],
                          [alpha * beta_k * S * math.pow(I1+I2, alpha-1) / N],
                          [-gamma1],
                          [gamma1],
                          [0]])

    part_f_I2 = np.array([[-alpha * beta_k * S * math.pow(I1+I2, alpha-1) / N],
                          [alpha * beta_k * S * math.pow(I1+I2, alpha-1) / N],
                          [0],
                          [-gamma2],
                          [gamma2]])

    part_f_R = np.array([[0],
                         [0],
                         [0],
                         [0],
                         [0]])

    # 5x5
    f_jacob = np.concatenate([part_f_S, part_f_E, part_f_I1, part_f_I2,
                              part_f_R], axis=1)

    # 5x5
    # P_k1_prior = f_jacob * P * f_jacob^T + Q
    P_k1_prior = np.matmul(np.matmul(f_jacob, P), np.transpose(f_jacob)) + Q
    return P_k1_prior


def update_step(x_hat, x_hat_k1, P_k1, Rn, rho1, rho2, z_k):
    # 5x5
    ep = 10**-8
    
    H = np.array([[ep, 0, 0, 0, 0],
                  [0, ep, 0, 0, 0],
                  [0, 0, ep, rho1, 0],
                  [0, 0, 0, rho2, 0],
                  [0, 0, 0, 0, ep]])
    
    
    # Si = H * P_k1 * H^T + Rn
    Si = np.matmul(np.matmul(H, P_k1), np.transpose(H)) + Rn

    # K_new = P_k1 * H^T * Si^(-1)
    K_new = np.matmul(np.matmul(P_k1, np.transpose(H)), np.linalg.inv(Si))
    y_new = np.matmul(H, x_hat)

    # 5x1
    diff = z_k - y_new

    x_hat_k1_post = x_hat_k1 + np.matmul(K_new, diff)

    P_k1_post = P_k1 - np.matmul(np.matmul(K_new, Si), np.transpose(K_new))


    return x_hat_k1_post, P_k1_post, diff


def get_data_sets(state, fips=None):
    # the post hoc seiir model predictions provided by Prof Flaxman without
    # any kalman filtering:
    if state == 'New York':
        filename = 'new_york'
    elif state == 'Florida':
        filename = 'florida'
            
    seiir = pd.read_csv(r'data/seiir_projections/' + filename + '_proj.csv', header=0,
                        index_col='date', parse_dates=True)
    

    fb_data = data_sets.create_symptom_df()
    fb_data_val = data_sets.create_symptom_df(valid=True)

    if fips is None:
        case_data = data_sets.create_case_df_state()
        case_data_geo = case_data.loc[state]['case_rate'].copy()
        fb_data_geo = fb_data.loc[state].groupby('date').sum().copy()
        fb_data_val_geo = fb_data_val.loc[state].groupby('date').sum().copy()

    else:
        case_data = data_sets.create_case_df_county()
        case_data_geo = case_data.loc[fips]['case_rate'].copy()
        
        if fips == 'New York City':
            nyc_fips = ['36005', '36061', '36047', '36085']
            fb_data_geo = fb_data.loc[(slice(None), '36081'), :].copy()
            fb_data_geo = fb_data_geo.mean(level='date')
            fb_data_val_geo = fb_data_val.loc[(slice(None), '36081'), :].copy()
            fb_data_val_geo = fb_data_val.mean(level='date')
            for borough in nyc_fips:
                fb_data_geo += fb_data.loc[(slice(None), borough), :].copy().mean(level='date')
                fb_data_val_geo += fb_data_val.loc[(slice(None), borough), :].copy().mean(level='date')
        else:
            fb_data_geo = fb_data.loc[(slice(None), fips), :].copy()
            fb_data_val_geo = fb_data_val.loc[(slice(None), fips), :].copy()
            # collapse down to a single index column (date)
            fb_data_geo = fb_data_geo.mean(level='date')
            fb_data_val_geo = fb_data_val_geo.mean(level='date')


    return seiir, fb_data_geo, fb_data_val_geo, case_data_geo


def calc_fb_ma7(fb_data):
    """
    Returns a Pandas series
    """
    # the fb_data is a DataFrame while the case_data is a Series
    fb_ma7 = fb_data.rolling(window=7).mean()
    fb_ma7 = fb_ma7.iloc[6:, :]
    prop_ma7 = fb_ma7['num_stl'].div(fb_ma7['n'])

    return prop_ma7, fb_ma7['n'].copy(), fb_ma7['num_stl'].copy()

In [5]:
# bring in new data
full_data = pd.read_csv(r'data/from_challenge/overall-county.csv', header=0, dtype={'fips': 'str', 'pct_hh_cli': 'float64'},
                        parse_dates=[0])
time_start = full_data['date'].min()
time_end = full_data['date'].max()
full_data.set_index(['fips', 'date'], inplace=True)
full_data.sort_index(inplace=True)

# derive count of survey respondents with household members having covid symptoms
full_data['num_hh_cli'] = full_data['n'].mul(full_data['pct_hh_cli']/100.).round()
full_data['num_hh_cli'] = full_data['num_hh_cli'].astype('int64')

# group by county and date
data_of_interest = full_data[['n', 'num_hh_cli']].copy().groupby(level=(0, 1)).sum()
idx = pd.IndexSlice

# create full date range
date_rng = pd.date_range(time_start, time_end)
iterables = [data_of_interest.index.levels[0], date_rng]
new_index = pd.MultiIndex.from_product(iterables, names=['fips', 'date'])

data_of_interest = data_of_interest.reindex(index=new_index)
# this will have NaN values in the new index entries for which there was no
# previous data -- I will fill them upon extracting a particular county




In [6]:
time_start

Timestamp('2020-04-06 00:00:00')

In [17]:
# set constants
#K0 = datetime.date(2020, 4, 12)
K0 = datetime.date(2020, 4, 18)

the_state = 'NY'
the_county = data_sets.get_fips(the_state, 'Westchester')
#the_county = 'New York City'
constants = {
    'alpha': 0.948786,
    'gamma1': 0.500000,
    'gamma2': 0.662215,
    'sigma': 0.266635,
    'theta': 6.000000
    }

# set initial values for Kalman filter parameters
P_mult = 1
Q_mult = 1

# Rn is the R noise covariance matrix; it remains constant thru the stepping of the
# Kalman filter
Rn_mult = 5*10**-8

Rn_22 = 10000
Rn_32 = 1000

Rn_23 = 1000
Rn_33 = 100

Rn = Rn_mult * np.array([[0, 0, 0, 0, 0],
                         [0, 0, 0, 0, 0],
                         [0, 0, Rn_22, Rn_23, 0],
                         [0, 0, Rn_32, Rn_33, 0],
                         [0, 0, 0, 0, 0]])

Q = Q_mult * np.eye(5)
P = P_mult * np.eye(5)

# generate data
seiir, fb_data, fb_data_val, case_data = get_data_sets(
    data_sets.STATES[the_state], fips=the_county)

idx = pd.IndexSlice
hh_cli = data_of_interest.loc[idx[the_county, :], :].loc[the_county].copy()
hh_cli.fillna(method='pad', inplace=True)



if the_county == 'New York City':
    county_pop = 0
    for each in ['36081', '36005', '36061', '36047', '36085']:
        this_count, state_pop = data_sets.get_pops(each)
        county_pop += this_count
else:
    county_pop, state_pop = data_sets.get_pops(the_county)

b = county_pop / state_pop

# calculate moving averages on the fb and case data
hh_cli_ma7 = hh_cli.rolling(window=7).mean()
hh_cli_ma7 = hh_cli_ma7.iloc[6:, :]
num_cli_ma7 = hh_cli_ma7['num_hh_cli']
prop_cli_ma7 = num_cli_ma7.div(hh_cli_ma7['n'])


case_ma7 = case_data.rolling(window=7).mean()
case_ma7_all = case_ma7.iloc[6:]
prop_ma7, n_ma7, num_stl_ma7 = calc_fb_ma7(fb_data)

prop_ma7_valid, n_ma7_valid, num_stl_ma7_valid = calc_fb_ma7(fb_data_val)

# get starting compartment values for the state level
x_hat_state_k0, beta_k0 = get_predicts_prior(K0, seiir)

# convert to the county level
x_hat_k0 = b * x_hat_state_k0

I2_county = x_hat_k0[3, 0]


rho1 = 0.0001
rho2 = case_ma7_all.loc['2020-04-12'] / I2_county


# create empty dictionaries to hold the estimated values
prop_est = {}
case_est = {}
seiir_pred = {}

diff_rat = {}



# Original data run ----------------
start = K0
d = start
#while d <= datetime.date(2020, 9, 30):
while d <= datetime.date(2020, 10, 23):    

# each cycle of the while loop executes a step

    # get state level compartments
    x_hat_state_k, beta_k = get_predicts_prior(d, seiir)

    # step the state level compartments 7 days forward
    x_hat_state_k1 = step_seiir(x_hat_state_k, constants, beta_k)
    
    # convert the state level compartments to county level values
    x_hat_k = b * x_hat_state_k
    x_hat_k1 = b * x_hat_state_k1
    
    indexDate = d + datetime.timedelta(days=7)
    # store seiir prediction before it's modified by Kalman filter
    seiir_pred[indexDate] = x_hat_k1[3, 0]

    # get measurements for current day
    z_k = np.array([[0],
                    [0],
                    [prop_ma7.loc[d - datetime.timedelta(days=6)]],
                    [case_ma7_all.loc[d]],
                    [0]])    
    
    """
    z_k = np.array([[0],
                    [0],
                    [prop_cli_ma7.loc[d - datetime.timedelta(days=6)]],
                    [case_ma7_all.loc[d]],
                    [0]])
    """

    # predict step using the stepped fwd SEIIR compartment values 
    P = predict_step(x_hat_k1, P, Q, beta_k, constants)
    print('step -----------')
    print('P:')
    print(P)

    # update step
    x_hat_post, P_post, the_diff = update_step(x_hat_k, x_hat_k1, P, Rn,
                                     rho1, rho2, z_k)
    print('P_post:')
    print(P_post)
    

    # store estimated values for proportion and case rate
    
    prop_est[indexDate] = rho1 * x_hat_post[3, 0]
    case_est[indexDate] = rho2 * x_hat_post[3, 0]
    
    
    diff_rat[indexDate] = the_diff[2, 0] / the_diff[3, 0]

    # update the P and d
    P = P_post
    d += datetime.timedelta(days=1)
    
# create pandas series of the estimated case rate
predicted_case = pd.Series(case_est)
predicted_seiir_prior = pd.Series(seiir_pred)

step -----------
P:
[[ 1.11253926 -0.11253926  0.11860326  0.03847845 -0.15708171]
 [-0.11253926  1.18363349 -0.18969748 -0.03847845  0.15708171]
 [ 0.11860326 -0.18969748  1.32109422 -0.25        0.        ]
 [ 0.03847845 -0.03847845 -0.25        1.68852871 -0.43852871]
 [-0.15708171  0.15708171  0.         -0.43852871  1.43852871]]
P_post:
[[-2.22044605e-16 -2.77555756e-17 -3.60822483e-16  2.74086309e-15
  -2.83106871e-15]
 [-1.38777878e-17 -4.44089210e-16 -5.55111512e-17  7.77156117e-16
  -3.33066907e-16]
 [-3.74700271e-16 -8.32667268e-17  1.24102413e+00  7.48097967e-09
  -2.36595627e-16]
 [ 2.73392420e-15  7.70217223e-16  7.48097967e-09 -2.22044605e-16
   1.11022302e-16]
 [-2.80331314e-15 -3.33066907e-16 -2.30882669e-16  1.11022302e-16
   0.00000000e+00]]
step -----------
P:
[[ 1.06948372e+00 -6.94837217e-02  1.46825555e-01 -1.46825554e-01
  -1.17221938e-09]
 [-6.94837217e-02  1.06948372e+00 -1.46825555e-01  1.46825554e-01
   1.17221925e-09]
 [ 1.46825555e-01 -1.46825555e-01  1.310

step -----------
P:
[[ 1.06093890e+00 -6.09388972e-02  1.35909851e-01 -1.35909850e-01
  -1.08507105e-09]
 [-6.09388972e-02  1.06093890e+00 -1.35909851e-01  1.35909850e-01
   1.08507104e-09]
 [ 1.35909851e-01 -1.35909851e-01  1.30311490e+00 -3.03114897e-01
  -2.41999538e-09]
 [-1.35909850e-01  1.35909850e-01 -3.03114897e-01  1.30311489e+00
   2.41999549e-09]
 [-1.08507104e-09  1.08507104e-09 -2.41999537e-09  2.41999547e-09
   1.00000000e+00]]
P_post:
[[ 6.66133815e-16 -3.26128013e-16  3.60822483e-16 -1.38777878e-15
  -1.44756607e-24]
 [-3.26128013e-16  2.22044605e-16 -5.55111512e-17  0.00000000e+00
   4.13590306e-25]
 [ 3.88578059e-16 -5.55111512e-17  1.21271330e+00  7.31031974e-09
   4.13590306e-25]
 [-1.41553436e-15  0.00000000e+00  7.31031979e-09  0.00000000e+00
  -8.27180613e-25]
 [-1.24077092e-24  2.06795153e-25  8.46504668e-18 -1.24077092e-24
   0.00000000e+00]]
step -----------
P:
[[ 1.06033485e+00 -6.03348541e-02  1.35248734e-01 -1.35248733e-01
  -1.07979285e-09]
 [-6.03348541e-

step -----------
P:
[[ 1.06762678e+00 -6.76267766e-02  1.43097844e-01 -1.43097843e-01
  -1.14245825e-09]
 [-6.76267766e-02  1.06762678e+00 -1.43097844e-01  1.43097843e-01
   1.14245815e-09]
 [ 1.43097844e-01 -1.43097844e-01  1.30279416e+00 -3.02794157e-01
  -2.41743455e-09]
 [-1.43097843e-01  1.43097843e-01 -3.02794157e-01  1.30279415e+00
   2.41743465e-09]
 [-1.14245826e-09  1.14245815e-09 -2.41743456e-09  2.41743467e-09
   1.00000000e+00]]
P_post:
[[ 0.00000000e+00 -1.80411242e-16  1.38777878e-16 -4.99600361e-16
   2.06795153e-24]
 [-1.66533454e-16  0.00000000e+00 -1.94289029e-16  7.77156117e-16
   2.06795153e-25]
 [ 1.11022302e-16 -1.94289029e-16  1.21055919e+00  7.29733463e-09
  -4.13590306e-25]
 [-4.99600361e-16  7.77156117e-16  7.29733463e-09  2.22044605e-16
   2.48154184e-24]
 [ 2.06795153e-24  2.06795153e-25 -3.96104783e-18  2.06795153e-24
   1.11022302e-16]]
step -----------
P:
[[ 1.06969901e+00 -6.96990061e-02  1.45236679e-01 -1.45236678e-01
  -1.15953421e-09]
 [-6.96990061e-

step -----------
P:
[[ 1.08428898e+00 -8.42889800e-02  1.59354800e-01 -1.59354799e-01
  -1.27224982e-09]
 [-8.42889800e-02  1.08428898e+00 -1.59354800e-01  1.59354799e-01
   1.27224982e-09]
 [ 1.59354800e-01 -1.59354800e-01  1.30127251e+00 -3.01272506e-01
  -2.40528616e-09]
 [-1.59354799e-01  1.59354799e-01 -3.01272506e-01  1.30127250e+00
   2.40528616e-09]
 [-1.27224983e-09  1.27224983e-09 -2.40528618e-09  2.40528617e-09
   1.00000000e+00]]
P_post:
[[ 0.00000000e+00  1.94289029e-16  2.77555756e-17 -1.66533454e-16
   6.20385459e-25]
 [ 1.94289029e-16  4.44089210e-16 -3.60822483e-16  1.58206781e-15
   1.24077092e-24]
 [ 0.00000000e+00 -3.33066907e-16  1.20496813e+00  7.26363131e-09
   0.00000000e+00]
 [-1.94289029e-16  1.58206781e-15  7.26363136e-09  0.00000000e+00
  -1.65436123e-24]
 [ 8.27180613e-25  1.24077092e-24 -1.25021389e-17 -1.24077092e-24
   1.11022302e-16]]
step -----------
P:
[[ 1.08434146e+00 -8.43414570e-02  1.59396336e-01 -1.59396335e-01
  -1.27258140e-09]
 [-8.43414570e-

step -----------
P:
[[ 1.07623377e+00 -7.62337705e-02  1.51681139e-01 -1.51681137e-01
  -1.21098516e-09]
 [-7.62337705e-02  1.07623377e+00 -1.51681139e-01  1.51681137e-01
   1.21098502e-09]
 [ 1.51681139e-01 -1.51681139e-01  1.30179758e+00 -3.01797582e-01
  -2.40947807e-09]
 [-1.51681137e-01  1.51681137e-01 -3.01797582e-01  1.30179758e+00
   2.40947821e-09]
 [-1.21098516e-09  1.21098502e-09 -2.40947807e-09  2.40947821e-09
   1.00000000e+00]]
P_post:
[[ 2.22044605e-16 -9.71445147e-17 -2.77555756e-17  3.60822483e-16
  -2.06795153e-24]
 [-8.32667268e-17 -1.11022302e-15  3.60822483e-16 -1.33226763e-15
  -1.86115638e-24]
 [-5.55111512e-17  3.88578059e-16  1.20752583e+00  7.27904970e-09
  -8.27180613e-25]
 [ 3.60822483e-16 -1.38777878e-15  7.27904970e-09  0.00000000e+00
   0.00000000e+00]
 [-2.27474668e-24 -1.86115638e-24  4.39126530e-18  8.27180613e-25
   1.11022302e-16]]
step -----------
P:
[[ 1.07518764e+00 -7.51876364e-02  1.50657736e-01 -1.50657734e-01
  -1.20281462e-09]
 [-7.51876364e-

...

step -----------
P:
[[ 1.15916093e+00 -1.59160928e-01  2.16683824e-01 -2.16683823e-01
  -1.72995079e-09]
 [-1.59160928e-01  1.15916093e+00 -2.16683824e-01  2.16683823e-01
   1.72995078e-09]
 [ 2.16683824e-01 -2.16683824e-01  1.29499627e+00 -2.94996265e-01
  -2.35517816e-09]
 [-2.16683823e-01  2.16683823e-01 -2.94996265e-01  1.29499626e+00
   2.35517807e-09]
 [-1.72995079e-09  1.72995079e-09 -2.35517817e-09  2.35517807e-09
   1.00000000e+00]]
P_post:
[[-2.22044605e-16  2.77555756e-17  8.32667268e-17 -7.21644966e-16
  -2.06795153e-25]
 [ 8.32667268e-17  0.00000000e+00  5.55111512e-17 -1.11022302e-16
   0.00000000e+00]
 [ 2.77555756e-17  8.32667268e-17  1.18285065e+00  7.13030568e-09
  -4.13590306e-25]
 [-6.93889390e-16 -1.38777878e-16  7.13030573e-09  0.00000000e+00
   0.00000000e+00]
 [-4.13590306e-25  0.00000000e+00 -4.24395270e-18 -4.13590306e-25
   1.11022302e-16]]
step -----------
P:
[[ 1.14780547e+00 -1.47805466e-01  2.09064459e-01 -2.09064457e-01
  -1.66911958e-09]
 [-1.47805466e-

step -----------
P:
[[ 1.08034332e+00 -8.03433207e-02  1.55603376e-01 -1.55603375e-01
  -1.24229937e-09]
 [-8.03433207e-02  1.08034332e+00 -1.55603376e-01  1.55603375e-01
   1.24229935e-09]
 [ 1.55603376e-01 -1.55603376e-01  1.30136184e+00 -3.01361834e-01
  -2.40599933e-09]
 [-1.55603375e-01  1.55603375e-01 -3.01361834e-01  1.30136183e+00
   2.40599934e-09]
 [-1.24229936e-09  1.24229934e-09 -2.40599931e-09  2.40599932e-09
   1.00000000e+00]]
P_post:
[[ 0.00000000e+00  0.00000000e+00  1.94289029e-16 -1.02695630e-15
  -1.24077092e-24]
 [ 0.00000000e+00 -2.22044605e-16  5.55111512e-17  0.00000000e+00
  -4.13590306e-25]
 [ 1.94289029e-16  8.32667268e-17  1.20612301e+00  7.27059307e-09
  -8.27180613e-25]
 [-1.02695630e-15  0.00000000e+00  7.27059307e-09  0.00000000e+00
  -4.13590306e-25]
 [-1.24077092e-24 -4.13590306e-25  1.25709793e-17 -8.27180613e-25
   1.11022302e-16]]
step -----------
P:
[[ 1.07947070e+00 -7.94706992e-02  1.54799418e-01 -1.54799417e-01
  -1.23588074e-09]
 [-7.94706992e-

step -----------
P:
[[ 1.10791729e+00 -1.07917286e-01  1.79915306e-01 -1.79915305e-01
  -1.43639987e-09]
 [-1.07917286e-01  1.10791729e+00 -1.79915306e-01  1.79915305e-01
   1.43639969e-09]
 [ 1.79915306e-01 -1.79915306e-01  1.29994747e+00 -2.99947472e-01
  -2.39470722e-09]
 [-1.79915305e-01  1.79915305e-01 -2.99947472e-01  1.29994747e+00
   2.39470740e-09]
 [-1.43639987e-09  1.43639969e-09 -2.39470722e-09  2.39470740e-09
   1.00000000e+00]]
P_post:
[[ 0.00000000e+00 -5.55111512e-17  1.94289029e-16 -4.71844785e-16
  -4.13590306e-25]
 [-6.93889390e-17 -6.66133815e-16 -2.49800181e-16  1.36002321e-15
   0.00000000e+00]
 [ 1.38777878e-16 -2.49800181e-16  1.19788298e+00  7.22092153e-09
  -8.27180613e-25]
 [-4.99600361e-16  1.33226763e-15  7.22092147e-09  0.00000000e+00
   0.00000000e+00]
 [-4.13590306e-25  2.06795153e-25 -4.13590306e-25  4.13590306e-25
   1.11022302e-16]]
step -----------
P:
[[ 1.11421685e+00 -1.14216845e-01  1.84944867e-01 -1.84944866e-01
  -1.47655463e-09]
 [-1.14216845e-

step -----------
P:
[[ 1.17989174e+00 -1.79891741e-01  2.30308030e-01 -2.30308029e-01
  -1.83872314e-09]
 [-1.79891741e-01  1.17989174e+00 -2.30308030e-01  2.30308029e-01
   1.83872314e-09]
 [ 2.30308030e-01 -2.30308030e-01  1.29485394e+00 -2.94853939e-01
  -2.35404189e-09]
 [-2.30308029e-01  2.30308029e-01 -2.94853939e-01  1.29485394e+00
   2.35404179e-09]
 [-1.83872314e-09  1.83872314e-09 -2.35404188e-09  2.35404179e-09
   1.00000000e+00]]
P_post:
[[-2.22044605e-16 -3.60822483e-16  5.55111512e-17 -3.05311332e-16
  -2.27474668e-24]
 [-3.88578059e-16  0.00000000e+00 -5.55111512e-17  8.32667268e-17
   2.06795153e-25]
 [ 5.55111512e-17 -8.32667268e-17  1.17819852e+00  7.10226233e-09
  -1.24077092e-24]
 [-3.33066907e-16  5.55111512e-17  7.10226239e-09  0.00000000e+00
   1.24077092e-24]
 [-1.86115638e-24  6.20385459e-25  4.21731170e-18  1.65436123e-24
   1.11022302e-16]]
step -----------
P:
[[ 1.18446342e+00 -1.84463419e-01  2.33095757e-01 -2.33095755e-01
  -1.86097960e-09]
 [-1.84463419e-

step -----------
P:
[[ 1.18280000e+00 -1.82800002e-01  2.31880465e-01 -2.31880463e-01
  -1.85127697e-09]
 [-1.82800002e-01  1.18280000e+00 -2.31880465e-01  2.31880463e-01
   1.85127696e-09]
 [ 2.31880465e-01 -2.31880465e-01  1.29413867e+00 -2.94138671e-01
  -2.34833137e-09]
 [-2.31880463e-01  2.31880463e-01 -2.94138671e-01  1.29413867e+00
   2.34833147e-09]
 [-1.85127697e-09  1.85127697e-09 -2.34833138e-09  2.34833147e-09
   1.00000000e+00]]
P_post:
[[-6.66133815e-16  2.22044605e-16 -4.16333634e-16  1.41553436e-15
   3.10192730e-24]
 [ 2.22044605e-16  2.22044605e-16  2.77555756e-17  5.55111512e-17
   4.13590306e-25]
 [-3.88578059e-16  2.77555756e-17  1.17721987e+00  7.09636322e-09
   4.13590306e-25]
 [ 1.47104551e-15  5.55111512e-17  7.09636327e-09  0.00000000e+00
   0.00000000e+00]
 [ 2.68833699e-24  4.13590306e-25 -8.43221671e-18  0.00000000e+00
   0.00000000e+00]]
step -----------
P:
[[ 1.18076657e+00 -1.80766573e-01  2.30652336e-01 -2.30652334e-01
  -1.84147199e-09]
 [-1.80766573e-

In [ ]:
diff_timeser = pd.Series(diff_rat)

In [ ]:
plt.plot(diff_timeser.index, diff_timeser)
plt.title('NYC')

In [ ]:
n = len(prop_ma7.values)
f_t = np.fft.fft(prop_ma7.values, n)

mag = np.sqrt(f_t * np.conj(f_t))
freq = (1/n)* np.arange(n)
L = np.arange(1, np.floor(n/2), dtype='int64')
plt.plot(freq[L], mag[L])

In [ ]:
prop_ma7.values

In [ ]:
# error computations

In [ ]:
# compute squared error for the overlapping time period -------------------------

left = predicted_seiir_prior.index[0]
right = d

seiir_err = case_ma7_all.loc[left:right] - predicted_seiir_prior[left:right]
seiir_sum_sq_err = (seiir_err**2).sum()
seiir_mse = seiir_sum_sq_err / seiir_err.count()

kalman_err = predicted_case.loc[left:right] - case_ma7_all.loc[left:right]
kalman_sum_sq_err = (kalman_err**2).sum()
kalman_mse = kalman_sum_sq_err / kalman_err.count()


print('SSE between seiir forecast and case rate:', seiir_sum_sq_err)
print('MSE between seiir forecast and measured case rate:', seiir_mse)
print()
print('SSE between kalman forecast and case rate:', kalman_sum_sq_err)
print('MSE between kalman forecast and measured case rate:', kalman_mse)

In [ ]:
# MASE calculation -----------------

left = predicted_case.index[0]
right = predicted_case.index[-1]

# take error between prediction and actual
e = (case_ma7_all.loc[left:right] - predicted_case.loc[left:right]).abs()

case_ma7_diff = case_ma7_all.diff(7)
denom = (case_ma7_diff.loc[left:right]).abs()

term = e.div(denom)
term.replace([np.inf, -np.inf], np.nan, inplace=True)
term.dropna(inplace=True)
mase = term.sum() / len(term)
print('mase:', mase)

In [11]:
output_df = pd.DataFrame()


In [18]:
the_county

'36119'

In [19]:
output_df[the_county] = predicted_case


In [21]:
output_df.to_csv(r'output/predicted_case_example.csv')

# All plotting code below this

In [ ]:
# Plotting constants and variables ----------------
matplotlib.rcParams.update({'font.size': 22})
plt.style.use('seaborn-whitegrid')
purple = '#33016F'
gold = '#9E7A27'
gray = '#797979'
width = 4
%matplotlib qt

tick_end = predicted_seiir_prior.index[-1]
special_start = datetime.date(2020, 8, 16)

week_interval = pd.date_range(start=special_start, end=tick_end, freq='W')
week_interval = [x.to_pydatetime().date() for x in week_interval]

In [ ]:
# Single plot with all lines
# Single plot----------

fig3, ax31 = plt.subplots(1)
plt.sca(ax31)
plt.plot(case_ma7_all.loc[special_start:d].index, case_ma7_all.loc[special_start:d], label='Confirmed Case Count', c=gold,
         linewidth=width)
plt.plot(predicted_seiir_prior.loc[special_start:tick_end].index, predicted_seiir_prior.loc[special_start:tick_end],
         label='IHME 7-day Forecast', c=gray, linewidth=width)
plt.plot(predicted_case.loc[special_start:tick_end].index, predicted_case.loc[special_start:tick_end], label='Our 7-Day Forecast',
         c=purple, linewidth=width)
plt.plot(case_ma7_all.loc[d:tick_end].index, case_ma7_all.loc[d:tick_end], c='darkorange',
         linewidth=width)
plt.ylim(-50, 1300)
#plt.ylim(-2, 72)
plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor')
# plt.xlabel('Date')
plt.xlim(special_start, datetime.date(2020, 11, 3))
plt.ylabel('Number of Cases per Day')
plt.legend(loc='upper left')

ax32 = ax31.twinx()
plt.sca(ax32)
plt.plot(num_stl_ma7.loc[special_start:d].index, num_stl_ma7.loc[special_start:d], c='red', label='Facebook Positive Symptoms, Smell/Taste Loss',
         linewidth=width)
plt.plot(num_stl_ma7.loc[d:tick_end].index, num_stl_ma7.loc[d:tick_end], c='maroon',
         linewidth=width)
#plt.plot(prop_ma7.index, num_stl_ma7, c='red', label='Facebook Positive Symptoms',
#         linewidth=width)
#plt.ylim(-.065, 2.5)
plt.ylim(-.4, 11)
plt.grid(axis='y', linestyle=':')
plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor')

plt.ylabel('Number of Positive Symptom Response per Day')
plt.legend(loc='upper right')

plt.show()

In [8]:
xlim_left = None
xlim_right = None

leftylim_low = None
leftylim_high = None

rightylim_low = None
rightylim_high = None

xtick_size = 14
xlabel_size = 14

In [9]:
# plot findings -- multiple plots

# Plotting constants and variables ----------------

plt.style.use('seaborn-whitegrid')
matplotlib.rcParams.update({'font.size': 18})
purple = '#33016F'
gold = '#9E7A27'
gray = '#797979'
width = 4
%matplotlib qt

tick_end = predicted_seiir_prior.index[-1]

week_interval = pd.date_range(start=start, end=tick_end, freq='W')
week_interval = [x.to_pydatetime().date() for x in week_interval]



In [ ]:
# multiple plots -----------
fig1, ax11 = plt.subplots(1)
plt.sca(ax11)
plt.plot(case_ma7_all.loc[start:d].index, case_ma7_all.loc[start:d], label='Confirmed Case Count', c=gold,
         linewidth=width)
plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor', fontsize=xtick_size)
plt.ylabel('Number of Cases per Day')
plt.legend(loc='upper left')
plt.ylim(leftylim_low, leftylim_high)
plt.xlim(xlim_left, xlim_right)


fig2, ax21 = plt.subplots(1)
plt.sca(ax21)
plt.plot(case_ma7_all.loc[start:d].index, case_ma7_all.loc[start:d], label='Confirmed Case Count', c=gold,
         linewidth=width)
plt.plot(predicted_seiir_prior.index, predicted_seiir_prior,
         label='IHME 7-day Forecast', c=gray, linewidth=width)
plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor', fontsize=xtick_size)
plt.ylabel('Number of Cases per Day')
plt.legend(loc='upper left')
plt.ylim(leftylim_low, leftylim_high)
plt.xlim(xlim_left, xlim_right)




fig3, ax31 = plt.subplots(1)
plt.sca(ax31)
plt.plot(case_ma7_all.loc[start:d].index, case_ma7_all.loc[start:d], label='Confirmed Case Count', c=gold,
         linewidth=width)
plt.plot(predicted_seiir_prior.index, predicted_seiir_prior,
         label='IHME 7-day Forecast', c=gray, linewidth=width)

ax32 = ax31.twinx()
plt.sca(ax32)
plt.plot(num_stl_ma7.loc[start:d].index, num_stl_ma7.loc[start:d], c='red', label='FB Positive Symptoms, Smell/Taste Loss',
         linewidth=width)
plt.grid(axis='y', linestyle=':')

plt.ylabel('Number of Positive Symptom Response per Day')
plt.legend(loc='upper right')
plt.ylim(rightylim_low, rightylim_high)


plt.sca(ax31)
plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor', fontsize=xtick_size)
plt.ylabel('Number of Cases per Day')

plt.legend(loc='upper left')
plt.ylim(leftylim_low, leftylim_high)
plt.xlim(xlim_left, xlim_right)






In [10]:
fig4, ax41 = plt.subplots(1)
plt.sca(ax41)
plt.plot(case_ma7_all.loc[start:d].index, case_ma7_all.loc[start:d], label='Confirmed Case Count', c=gold,
         linewidth=width)
plt.plot(predicted_seiir_prior.index, predicted_seiir_prior,
         label='IHME 7-day Forecast', c=gray, linewidth=width)
#naieve estimate
naive_start = start + datetime.timedelta(days=7)
naive_d = d + datetime.timedelta(days=7)
plt.plot(case_ma7_all.loc[naive_start:naive_d].index, case_ma7_all.loc[start:d], label='Naive guess', c='orange')

ax42 = ax41.twinx()
plt.sca(ax42)
plt.plot(num_stl_ma7.loc[start:d].index, num_stl_ma7.loc[start:d], c='red', label='FB Positive Symptoms, Smell/Taste Loss',
         linewidth=width)
plt.grid(axis='y', linestyle=':')

plt.ylabel('Number of Positive Household Symptom Response per Day')
plt.legend(loc='upper right')
plt.ylim(rightylim_low, rightylim_high)


plt.sca(ax41)
plt.plot(predicted_case.loc[start:].index, predicted_case.loc[start:], label='Our 7-Day Forecast',
         c=purple, linewidth=width)

plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor', fontsize=xtick_size)
plt.ylabel('Number of Cases per Day')

plt.legend(loc='upper left')
plt.ylim(leftylim_low, leftylim_high)
plt.xlim(xlim_left, xlim_right)

(737523.2, 737738.8)

In [ ]:
b


In [ ]:
plt.show()

In [ ]:
## special end date

# plot findings -- multiple plots

# Plotting constants and variables ----------------
matplotlib.rcParams.update({'font.size': 18})
plt.style.use('seaborn-whitegrid')
purple = '#33016F'
gold = '#9E7A27'
gray = '#797979'
width = 4
%matplotlib qt

tick_end = predicted_seiir_prior.index[-1]

week_interval = pd.date_range(start=start, end=tick_end, freq='W')
week_interval = [x.to_pydatetime().date() for x in week_interval]


fig1, ax11 = plt.subplots(1)
plt.sca(ax11)
plt.plot(case_ma7_all.loc[start:tick_end].index, case_ma7_all.loc[start:tick_end], label='Confirmed Case Count', c=gold,
         linewidth=width)
plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor', fontsize=xtick_size)
plt.ylabel('Number of Cases per Day')
plt.legend(loc='upper left')
plt.ylim(leftylim_low, leftylim_high)
plt.xlim(xlim_left, xlim_right)


fig2, ax21 = plt.subplots(1)
plt.sca(ax21)
plt.plot(case_ma7_all.loc[start:tick_end].index, case_ma7_all.loc[start:tick_end], label='Confirmed Case Count', c=gold,
         linewidth=width)
plt.plot(predicted_seiir_prior.index, predicted_seiir_prior,
         label='IHME 7-day Forecast', c=gray, linewidth=width)
plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor', fontsize=xtick_size)
plt.ylabel('Number of Cases per Day')
plt.legend(loc='upper left')
plt.ylim(leftylim_low, leftylim_high)
plt.xlim(xlim_left, xlim_right)




fig3, ax31 = plt.subplots(1)
plt.sca(ax31)
plt.plot(case_ma7_all.loc[start:tick_end].index, case_ma7_all.loc[start:tick_end], label='Confirmed Case Count', c=gold,
         linewidth=width)
plt.plot(predicted_seiir_prior.index, predicted_seiir_prior,
         label='IHME 7-day Forecast', c=gray, linewidth=width)

ax32 = ax31.twinx()
plt.sca(ax32)
plt.plot(num_stl_ma7.loc[start:tick_end].index, num_stl_ma7.loc[start:tick_end], c='red', label='FB Positive Symptoms, Smell/Taste Loss',
         linewidth=width)
plt.grid(axis='y', linestyle=':')

plt.ylabel('Number of Positive Symptom Response per Day')
plt.legend(loc='upper right')
plt.ylim(rightylim_low, rightylim_high)


plt.sca(ax31)
plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor', fontsize=xtick_size)
plt.ylabel('Number of Cases per Day')

plt.legend(loc='upper left')
plt.ylim(leftylim_low, leftylim_high)
plt.xlim(xlim_left, xlim_right)



fig4, ax41 = plt.subplots(1)
plt.sca(ax41)
plt.plot(case_ma7_all.loc[start:tick_end].index, case_ma7_all.loc[start:tick_end], label='Confirmed Case Count', c=gold,
         linewidth=width)
plt.plot(predicted_seiir_prior.index, predicted_seiir_prior,
         label='IHME 7-day Forecast', c=gray, linewidth=width)

ax42 = ax41.twinx()
plt.sca(ax42)
plt.plot(num_stl_ma7.loc[start:tick_end].index, num_stl_ma7.loc[start:tick_end], c='red', label='FB Positive Symptoms, Smell/Taste Loss',
         linewidth=width)
plt.grid(axis='y', linestyle=':')

plt.ylabel('Number of Positive Symptom Response per Day')
plt.legend(loc='upper right')
plt.ylim(rightylim_low, rightylim_high)


plt.sca(ax41)
plt.plot(predicted_case.loc[start:].index, predicted_case.loc[start:], label='Our 7-Day Forecast',
         c=purple, linewidth=width)

plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor', fontsize=xtick_size)
plt.ylabel('Number of Cases per Day')

plt.legend(loc='upper left')
plt.ylim(leftylim_low, leftylim_high)
plt.xlim(xlim_left, xlim_right)


In [ ]:
# NYC plot settings

xlim_left = datetime.date(2020, 4, 2)
xlim_right = datetime.date(2020, 11, 3)

leftylim_low = -700
leftylim_high = 12000

rightylim_low = -5
rightylim_high = 85

xtick_size = 14
xlabel_size = 14

In [ ]:
# Nassau plot settings
xlim_left = datetime.date(2020, 4, 2)
xlim_right = datetime.date(2020, 11, 3)

leftylim_low = -40
leftylim_high = 1650

rightylim_low = -.3
rightylim_high = 12

xtick_size = 14
xlabel_size = 14

In [ ]:
# Westchester plot settings
xlim_left = datetime.date(2020, 4, 2)
xlim_right = datetime.date(2020, 11, 3)

leftylim_low = -28
leftylim_high = 1000

rightylim_low = -.2
rightylim_high = 7

xtick_size = 14
xlabel_size = 14

In [ ]:
# Albany plot settings
xlim_left = datetime.date(2020, 4, 2)
xlim_right = datetime.date(2020, 11, 3)

leftylim_low = -3
leftylim_high = 75

rightylim_low = -.125
rightylim_high = 3.1

xtick_size = 14
xlabel_size = 14

In [ ]:
# Erie plot settings
xlim_left = datetime.date(2020, 4, 2)
xlim_right = datetime.date(2020, 10, 13)

leftylim_low = -10
leftylim_high = 250

rightylim_low = -.5
rightylim_high = 12.5

xtick_size = 14
xlabel_size = 14